In [ ]:
## script modeled after this, to improve performance by lumping together many lines into a single trace:
#https://plotly.com/python/lines-on-maps/#performance-improvement-put-many-lines-in-the-same-trace
    
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import json

mapbox_access_token = open(".mapbox_token").read()

In [ ]:
## note: in the tornadoes file, I manually made a fix to the Windsor tornado track. 
## In the original survey it was 2 separate tracks (https://www.ncdc.noaa.gov/stormevents/eventdetails.jsp?id=93763 and 
## https://www.ncdc.noaa.gov/stormevents/eventdetails.jsp?id=126020) but in the SPC database, it was a single tornado (which seems correct)
## but the single track didn't reflect the path through Windsor, so I created a 2nd entry with the same details, but the track split. 
## There are now two tracks, with a slight dogleg left in the middle

torn = pd.read_csv('1955-2018_all_tornadoes_CO.csv')
hail = pd.read_csv('1955-2018_hail_CO.csv')
wind = pd.read_csv('1955-2018_wind_CO.csv')

## where the end lat/lon is zero, set to the start lat/lon
torn['elat'] = torn['elat'].mask((torn['elat'] == 0.0), torn['slat'], axis=0)
torn['elon'] = torn['elon'].mask((torn['elon'] == 0.0), torn['slon'], axis=0)

## hail
hail['elat'] = hail['elat'].mask((hail['elat'] == 0.0), hail['slat'], axis=0)
hail['elon'] = hail['elon'].mask((hail['elon'] == 0.0), hail['slon'], axis=0)

## wind
wind['elat'] = wind['elat'].mask((wind['elat'] == 0.0), wind['slat'], axis=0)
wind['elon'] = wind['elon'].mask((wind['elon'] == 0.0), wind['slon'], axis=0)

## add a column with the text we want to use
torn['text'] = torn['date'].astype(str) + '<br>EF-' + torn['mag'].astype(str)
hail['text'] = hail['date'].astype(str) + '<br>size: ' + hail['mag'].astype(str) + "in"
wind['text'] = wind['date'].astype(str) + '<br>gust: ' + wind['mag'].astype(int).astype(str) + "kt"

## also read counties json file
with open('co_counties.geojson') as f:
    co_counties = json.load(f)


In [ ]:
## define function for pulling out the lat/lon/text arrays we'll need while going through:

def pull_lat_lon_text(df_sub):
    lons = []
    lats = []
    lons = np.empty(3 * len(df_sub))
    lons[::3] = df_sub['slon']
    lons[1::3] = df_sub['elon']
    lons[2::3] = None
    lats = np.empty(3 * len(df_sub))
    lats[::3] = df_sub['slat']
    lats[1::3] = df_sub['elat']
    lats[2::3] = None
    ## need to create similar arrays of text for each endpoint of the line
    texts = np.empty(3 * len(df_sub), dtype='object')
    texts[::3] = df_sub['text'].astype(str)
    texts[1::3] = df_sub['text'].astype(str)
    texts[2::3] = None
    return lats, lons, texts

In [ ]:
fig = go.Figure()

## will generate separate traces for non-sig and sig versions of each hazard

## hail 1-2"
df_sub = hail[(hail['mag']>=1) & (hail['mag']<2)]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 1.5,color = 'rgb(0,150,0)'),
    marker=go.scattermapbox.Marker(
        size=5,
        color='rgb(0,150,0)',
        opacity=1
    ),
    name = 'hail 1-2"',
)
)

## sig hail
df_sub = hail[(hail['mag']>=2)]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 4.5,color = 'rgb(0,150,0)'),
    marker=go.scattermapbox.Marker(
        size=9,
        color='rgb(0,150,0)',
        opacity=1
    ),
    name = 'hail 1-2"',
)
)

## wind 50-64 kt
df_sub = wind[(wind['mag']>=50) & (wind['mag']<65)]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 1.5,color = 'rgb(0,0,255)'),
    marker=go.scattermapbox.Marker(
        size=5,
        color='rgb(0,0,255)',
        opacity=1
    ),
    name = 'wind 50-64 kt',
)
)

## wind 65+ kt
df_sub = wind[(wind['mag']>=65)]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 4.5,color = 'rgb(0,0,255)'),
    marker=go.scattermapbox.Marker(
        size=9,
        color='rgb(0,0,255)',
        opacity=1
    ),
    name = 'wind 65+ kt',
)
)

## tornadoes < EF2
df_sub = torn[torn['mag']<2]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 1.5,color = 'red'),
    marker=go.scattermapbox.Marker(
        size=5,
        color='rgb(255,0,0)',
        opacity=0.85
    ),
    name = 'tornadoes EF<2',
)
)

## tornadoes > EF2
df_sub = torn[torn['mag']>=2]
lats,lons,texts = pull_lat_lon_text(df_sub)

fig.add_trace(go.Scattermapbox(
    lon = lons,
    lat = lats,
    hovertemplate ='%{text}<extra></extra>',
    text = list(texts),
    mode = 'markers+lines',
    line = dict(width = 4.5,color = 'red'),
    marker=go.scattermapbox.Marker(
        size=8,
        color='rgb(255,0,0)',
        opacity=1
    ),
    name = 'tornadoes EF2+',
)
)


## finally do the layout
fig.update_layout(
    title_text = 'Colorado severe weather reports, 1955-2018',
    title_x = 0.5,
    showlegend = True,
    width=1200,
    height=771,
    hovermode = 'closest',
    margin=dict(
        l=1,
        r=0,
        b=0,
        t=50,
        pad=1
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39.0,
            lon=-106.
        ),
        pitch=0,
        zoom=6.25,
        layers=[
            dict(
                sourcetype='geojson',
                source=co_counties,
                type='line',
               color='rgba(150,150,150,0.5)'
            ),
        ],
    ),
    annotations=[dict(
        x=0,
        y=0.01,
        xref="paper",
        yref="paper",
        text="Russ Schumacher/Colorado Climate Center/CSU<br>Source: https://www.spc.noaa.gov/wcm/#data",
        showarrow=False,
                 )],
)

fig.show()



In [ ]:
## if desired, write the map you created to html to display on a web page!
fig.write_html("severe_wx_CO_plotly_mapbox.html")